# Momentum Budget closure in MOM6
*Contributors:* [Hemant Khatri](https://github.com/hmkhatri), [Stephen Griffies](https://github.com/StephenGriffies)

Here, we outline the terms required to close the momentum budget in MOM6. Simialr to tracer budgets, the momentum budget closes on the native grid as well as on the remapped diagnostic grid ($z^*$ and $\rho_2$).

## Governing equations

As discussed in Appendix A to [Adcroft et al. 2019]( https://doi.org/10.1029/2019MS001726), MOM6 discretizes the vector-invariant velocity equation. This equation takes on the following continuous space-time form using a generalized vertical coordinate $s = s(x,y,z,t)$

$$
\left[ \frac{\partial \mathbf{u}}{\partial t} \right]_{s} =
- \left[ \frac{f + \zeta}{h} \right] \hat{\mathbf{z}} \wedge (h \, \mathbf{u}) 
- w^{(\dot{s})} \, \frac{\partial \mathbf{u}}{\partial z} -\left[ \rho^{-1} \, \nabla_s \, p + \nabla_s \Phi \right]
- \nabla_{s} K 
+ \boldsymbol{F}^{{\text{horz frict}}}
+ \boldsymbol{F}^{{\text{vert frict}}}
+ \rho^{-1} \, \boldsymbol{\tau}^{\text{bound}} 
\label{eq:continuous-velocity-equation}
$$

where we have 

\begin{alignat}{2}
\boldsymbol{v} &= \boldsymbol{u} + \hat{\boldsymbol{z}} \, w
= \hat{\boldsymbol{x}} \, u  +  \hat{\boldsymbol{y}} \, v
 + \hat{\boldsymbol{z}} \, w
&\mbox{velocity}
\\
\nabla_{s} &= 
\hat{\boldsymbol{x}} \left[ \frac{\partial}{\partial x} \right]_{s}
+
\hat{\boldsymbol{y}} \left[ \frac{\partial}{\partial y} \right]_{s}
&\mbox{horizontal gradient on $s$-surface}
\\
w^{(\dot{s})} &= \frac{\partial z}{\partial s} \, \frac{\mathrm{D}s}{\mathrm{D}t}
&\mbox{dia-surface velocity used for remapping}
\\
\zeta &= \left[ \frac{\partial v}{\partial x} \right]_{s} 
        -\left[ \frac{\partial u}{\partial y} \right]_{s} 
&\mbox{$s$-coordinate vertical vorticity}
\\
\Phi &= g \, z \qquad 
&\mbox{geopotential (more complex when have tides)}
\\
-\rho^{-1} \, \nabla_{z}p  &= 
-\left[ \rho^{-1} \, \nabla_s \, p + \nabla_s \Phi \right]
\qquad &\mbox{horizontal pressure acceleration} 
\\
K &= \frac{u^{2} + v^{2}}{2}  \qquad 
&\mbox{horizontal kinetic energy per unit mass}
\\
\boldsymbol{F}^{{\text{horz frict}}}&= 
\mbox{horiz friction from horz shear} &\mbox{Laplacian or biharmonic}
\\
\boldsymbol{F}^{{\text{vert frict}}} &= 
\mbox{vert friction from vertical shear} &\mbox{Laplacian}
\\
\rho^{-1} \, \boldsymbol{\tau}^{{\text{bound}}} &=
\mbox{boundary frictional acceleration} &\mbox{wind, bottom drag, etc.}
\end{alignat}

The hydrostatic pressure, $p$, arises from the weight per horizontal area above a point in the fluid, and it has a boundary condition at the sea surface, $z=\eta(x,y,t)$, set by the pressure applied by the overlying atmosphere and ice, $p(z=\eta)=p_{\text{\applied}}$. The external quantities include the vertical component of planetary vorticity $f=2\Omega \sin \phi$ and the gravitational potential $\Phi = g \, z$.  Finally, $\boldsymbol{F}$ is the acceleration due to the divergence of stresses including those provided through boundary interactions.

## Diagnosing terms in the discrete velocity equation

In this section we document how to diagnose terms in the MOM6 velocity equation. The velocity equation shown above is discretized in MOM6 by assuming the variables are constant within a grid cell layer according to 

\begin{equation}
 \boldsymbol{u} \rightarrow \boldsymbol{u}_{k}
 \qquad 
 h \rightarrow h_{k}
 \qquad 
 \zeta \rightarrow \zeta_{k}
 \qquad 
 p \rightarrow p_{k}
 \qquad 
 \mbox{etc.,}
\end{equation}

where $k$ is a cell index that increases downward. MOM6 then provides particular space and time discretizations of accelerations from the local time derivative, Coriolis, nonlinear vorticity, remapping, pressure, friction, boundaries. We are not concerned here with details for how the terms are discretized. Rather, we articulate the diagnostic terms required to close the velocity budget over a grid cell, and to identify where further diagnostics need to be coded. The table below summarizes these diagnostics and the tasks needed to close the budget.

| Process | Term  | Diagnostic Name   | 
| :----------------- | :----------------- | :----------------- | 
| Velocity tendency | $\left[ \frac{\partial \mathbf{u}}{\partial t} \right]_{s}$ | `dudt, dvdt` |
| Coriolis and kinetic energy gradient acceleration| $-\left[ \frac{f + \zeta}{h} \right] \hat{\mathbf{z}} \wedge (h\,\mathbf{u})- \nabla_{s}K $ | `CAu, CAv` | 
| Nonlienar Coriolis acceleration| $- \left[ \frac{\zeta}{h} \right] \hat{\mathbf{z}} \wedge (h \, \mathbf{u})$ | `rvxv, rvxu` | 
| Kinetic energy gradient acceleration| $- \nabla_{s} K$ | `gKEu, gKEv` | 
| Pressure acceleration (baroclinic contribution) | $-\left[ \rho^{-1} \, \nabla_s \, p + \nabla_s \Phi \right]^{\text{baroclinic}}$ | `PFu, PFv` | 
| Pressure acceleration (barotropic contribution) | $-\left[ \rho^{-1} \, \nabla_s \, p + \nabla_s \Phi \right]^{\text{barotropic}}$ | `u_BT_accel, v_BT_accel` | 
| Horizontal friction acceleration| $\boldsymbol{F}^{{\text{horz frict}}}$ | `diffu, diffv` | 
| Verical friction and boundary stress acceleration| $\boldsymbol{F}^{{\text{vert frict}}} + \rho^{-1} \, \boldsymbol{\tau}^{\text{bound}}$ | `du_dt_visc, dv_dt_visc` | 
| Boundary forcing                   | `boundary_forcing_heat_tendency` | `boundary_forcing_salt_tendency` | 
 
The sign convention is such that the grid-cell time tendency is equal to the sum of all the other terms. For example, in the case of zonal momentum equation

```dudt = CAu + PFu + u_BT_accel + diffu + du_dt_visc``` + residual due to vertical remapping


### Note
1. An online diagnostic for vertical remapping term $w^{(\dot{s})} \, \frac{\partial \mathbf{u}}{\partial z}$ is currently not available in MOM6. Nethertheless, this term is expected to be small in comparison to other diagnostic terms. We confirm this when we anlyze plots of momentum budget terms later in this tutorial. 
2. The contribution due to linear Coriolis term $- \left[ \frac{f}{h} \right] \hat{\mathbf{z}} \wedge (h \, \mathbf{u})$ can be computed as  `CAu - rvxv - gKEu`, `CAv - rvxu - gKEv`.
3. Two separate diagnostics are saved for pressure accelaration term (barotropic and baroclinic) because MOM6 uses split scheme for integrating the velocity equation in time. The net pressure force accelaration is the sum of these two diagnostics.
4. Contribution from vertical friction, surface wind stress and bottom friction are combined into one single diagnostic. It is tricky to compute individual components. We discuss this issue in more detail later when we look at depth-averaged momentum budgets.